# [python/Tensorflow] TFRecord를 만들어서 최소한의 CIFAR-10데이터로 학습시키기
- https://engineer-mole.tistory.com/212

### 참고

- 텐서플로우 Dataset: repeat(), batch(), take()
    - https://deep-deep-deep.tistory.com/27


In [6]:
import tensorflow as tf 

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    tf.config.experimental.set_visible_devices(gpus[4], 'GPU')
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

# 3. NumPy 배열의 TFRecord화

In [8]:

import numpy as np 

def _bytes_feature(value): 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value])) 
def serialize_sample(image, label): 
    image_binary = (image.astype(np.float32) / 255.0).tobytes() 
    label_binary = np.eye(10).astype(np.float32)[label].tobytes() 
    image_list = _bytes_feature(image_binary) 
    label_list = _bytes_feature(label_binary) 
    proto = tf.train.Example(features=tf.train.Features(feature={ "image": image_list, # float32, (32, 32, 3) 
                                                                 "label": label_list # float32, (10, ) 
                                                                })) 
    return proto.SerializeToString() 

def write_record(): 
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data() 
    
    print("X_train shape: ", X_train.shape)
    print("y_train shape: ", y_train.shape)

    print("X_test shape: ", X_test.shape)
    print("y_test shape: ", y_test.shape)
    
    
    with tf.io.TFRecordWriter("train.tfrecord") as writer: 
        for i in range(X_train.shape[0]): 
            example = serialize_sample(X_train[i], y_train[i]) 
            writer.write(example) 
    with tf.io.TFRecordWriter("test.tfrecord") as writer: 
        for i in range(X_test.shape[0]): 
            example = serialize_sample(X_test[i], y_test[i]) 
            writer.write(example) 
                    
if __name__ == "__main__": 
    write_record()



X_train shape:  (50000, 32, 32, 3)
y_train shape:  (50000, 1)
X_test shape:  (10000, 32, 32, 3)
y_test shape:  (10000, 1)


# 4. 만든 TFRecord를 읽어들이기

In [11]:
import tensorflow as tf 
import tensorflow.keras.layers as layers 
import numpy as np 

def deserialize_example(serialized_string): 
    image_feature_description = { 'image': tf.io.FixedLenFeature([], tf.string), 
                                 'label': tf.io.FixedLenFeature([], tf.string), } 
    example = tf.io.parse_single_example(serialized_string, image_feature_description) 
    image = tf.reshape(tf.io.decode_raw(example["image"], tf.float32), (32, 32, 3)) 
    label = tf.io.decode_raw(example["label"], tf.float32) 
    return image, label 

def read_record(): 
    dataset = tf.data.TFRecordDataset("train.tfrecord").map(deserialize_example).batch(1) 
    for x in dataset: 
        print(x) 
        break 

def read_record2(): 
#     dataset = tf.data.TFRecordDataset("train.tfrecord").map(deserialize_example).batch(1) 
    dataset = tf.data.TFRecordDataset("train.tfrecord").map(deserialize_example)    
    ds_size = sum(1 for _ in dataset)    
    print("# of batches loading TFRecord : {0}".format(tf.data.experimental.cardinality(dataset).numpy()))    
    print("# of batches loading TFRecord : {0}".format(ds_size)) 

            
if __name__ == "__main__": 
    read_record2()



# of batches loading TFRecord : -2
# of batches loading TFRecord : 50000


# 5. TFRecord로 학습시키기

In [2]:
import tensorflow as tf 

import tensorflow.keras.layers as layers 
import numpy as np 
# gpus = tf.config.experimental.list_physical_devices('GPU') 
# if gpus: 
#     try: # Currently, memory growth needs to be the same across GPUs 
#         for gpu in gpus: 
#             tf.config.experimental.set_memory_growth(gpu, True) 
#             logical_gpus = tf.config.experimental.list_logical_devices('GPU') 
#             print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs") 
#     except RuntimeError as e: # Memory growth must be set before GPUs have been initialized 
#         print(e) 
        
def deserialize_example(serialized_string): 
    image_feature_description = { 'image': tf.io.FixedLenFeature([], tf.string), 
                                 'label': tf.io.FixedLenFeature([], tf.string), } 
    example = tf.io.parse_single_example(serialized_string, image_feature_description) 
    image = tf.reshape(tf.io.decode_raw(example["image"], tf.float32), (32, 32, 3)) 
    label = tf.io.decode_raw(example["label"], tf.float32) 
    
    return image, label 

def conv_bn_relu(inputs, chs): 
    x = layers.Conv2D(chs, 3, padding="same")(inputs) 
    x = layers.BatchNormalization()(x) 
    
    return layers.ReLU()(x) 

def create_model(): 
    inputs = layers.Input((32, 32, 3)) 
    x = inputs 
    for chs in [64, 128, 256]: 
        for i in range(3): 
            x = conv_bn_relu(x, chs) 
        x = layers.AveragePooling2D(2)(x) 
    x = layers.GlobalAveragePooling2D()(x) 
    x = layers.Dense(10, activation="softmax")(x) 
    return tf.keras.models.Model(inputs, x) 

def main(): 
    trainset = tf.data.TFRecordDataset("train.tfrecord").map(deserialize_example).shuffle(2048).repeat().batch(128) 
    testset = tf.data.TFRecordDataset("test.tfrecord").map(deserialize_example).batch(128) 
    model = create_model() 
    model.compile("adam", "categorical_crossentropy", ["accuracy"]) 
    model.fit(trainset, steps_per_epoch=50000//128, validation_data=testset, epochs=3) 
    
if __name__ == "__main__": 
    main()



Physical devices cannot be modified after being initialized
[2021-10-02 14:26:29.127 ip-172-16-67-78:105296 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-10-02 14:26:29.532 ip-172-16-67-78:105296 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Train for 390 steps
Epoch 1/3
390/390 [==============================] - 12s 30ms/step - loss: 1.2954 - accuracy: 0.5295 - val_loss: 2.0364 - val_accuracy: 0.3797
Epoch 2/3
390/390 [==============================] - 8s 20ms/step - loss: 0.8036 - accuracy: 0.7151 - val_loss: 1.2064 - val_accuracy: 0.6024
Epoch 3/3
390/390 [==============================] - 8s 20ms/step - loss: 0.6140 - accuracy: 0.7872 - val_loss: 1.0648 - val_accuracy: 0.6521
